# Models

Define our project:

In [ ]:
stty -echo
mkdir workspace4
django-admin startproject myproject workspace4
cd workspace4

Create our webapp:

In [ ]:
python manage.py startapp models_app

Django uses a Model-View architecture; we will modify `view.py` and `models.py` in this tutorial.  
We begin by defining a model:

In [ ]:
cat << EOF > models_app/models.py
from django.db import models

class Match(models.Model):
    homeTeam = models.CharField(max_length=20)
    homeScore = models.IntegerField(default=0)
    awayTeam = models.CharField(max_length=20)
    awayScore = models.IntegerField(default=0)

    def play(self, home, score1, away, score2):
        self.homeTeam = home
        self.homeScore = score1
        self.awayTeam = away
        self.awayScore = score2
        
    def __str__(self):
        return f"{self.homeTeam} {self.homeScore}-{self.awayScore} {self.awayTeam}"
EOF

Now perform migrations for the default apps

In [ ]:
python manage.py migrate

The `migrate` command has created a new class in `models/apps.py` 

In [ ]:
cat models_app/apps.py

`makemigrations` basically generates the SQL commands for preinstalled apps (which can be viewed in installed apps in settings.py) and your newly created apps' model which you add in installed apps.It does not execute those commands in your database file. So tables aren't created after makemigrations.

After applying makemigrations you can see those SQL commands with `sqlmigrate` which shows all the SQL commands that have been generated by makemigrations.

`migrate` executes those SQL commands in database file. So after executing migrate, all the tables of your installed apps are created in your database file.  

However, our model has not yet been registered, so `the table corresponding to our model has not yet been created`.  

This `ModelsConfig` class has to be published in the global `project/settings.py` file.  

In [ ]:
sed -i -e "/INSTALLED_APPS/ a \    'models_app.apps.ModelsAppConfig'," myproject/settings.py

The relevant part of this file now reads:

In [ ]:
sed -n '/^INSTALLED_APPS/,/^]/p' myproject/settings.py

Now tell Django that you’ve made some changes to your models (in this case, you’ve made new ones) and that you’d like the changes to be stored as a migration:

In [ ]:
python manage.py makemigrations models_app

Let's take a moment to see the migrations Django has created:

In [ ]:
python manage.py sqlmigrate models_app 0001

Now, run migrate to create the `models_app_match` table in your database:

In [ ]:
python manage.py migrate

Check that the new table has been created using `dbshell`:

In [ ]:
python manage.py dbshell << EOF
SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name = 'models_app_match';
EOF

Use django shell to add data to the `models_app_match` table

In [ ]:
# Access model field values via Python attributes.
python manage.py shell << EOF
from models_app.models import Match  # Import the model classes we just wrote.

def playMatch(home, score1, away, score2):
    m = Match()
    m.play(home, score1, away, score2)
    m.save()

playMatch("Red", 2, "Blue", 1)
playMatch("Green", 5, "White", 0)
playMatch("Red", 3, "Green", 1)

for match in Match.objects.all():
    print(match)
EOF

Now use `dbshell` to see the same data:

In [ ]:
python manage.py dbshell << EOF
SELECT 
    *
FROM 
    models_app_match
EOF

Now it's time to create views so that we can see our results in a browser.

In [ ]:
cat << EOF >> models_app/views.py
from models_app.models import Match


def resultsView(request):
    matches = []
    try:
        for match in Match.objects.all():
            matches.append(match)
    except:
        raise Http404("Problems!!")
    
    context = {
        'matches':matches, 
    }
    return render(request, "resultsTemplate.html", context)
EOF

Now create the template

In [ ]:
mkdir -p models_app/templates
cat << EOF > models_app/templates/resultsTemplate.html
<style>
body {background-color: powderblue;}
table, th, td {
  border: 1px solid;
}
</style>

<h1>Results</h1>
<table>
{% for match in matches %}
    <tr><td>{{ match }}</td></tr>
{% endfor %}
</table>
EOF

Set up the `urls.py` to direct us to the view:

In [ ]:
# add url pattern to point at a new view
cat << EOF > models_app/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('results/', views.resultsView, name='resultsView'),
]
EOF

Now modify the master urls file:

In [ ]:
cat << EOF > myproject/urls.py
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('models_app/', include('models_app.urls')),
]
EOF

Let's use `firefox` to see the app:

In [ ]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

In [ ]:
firefox http://localhost:7000/models_app/results

clean up:
* kill the server
* remove workspace

In [ ]:
cd ..
fuser -k 7000/tcp
rm -r workspace4